# Exercise 4

In [ ]:
/* enerate adult_int91 = post_hike91 * med_wage_oth
generate teen_int91 = post_hike91 * med_wage_teen*/

#### Plan:

Use Angrist and Krueger's "Birth Quarter" data on the 1940s birth cohorts in the 1980 Census.

Produce OLS and 2SLS estimates of the effect of schooling in a log-wage regression.

Replicate Angrist and Krueger's Table 6. Use regress and ivregress 2sls.

Compare the Estimates. How does the Wald estimate in Part C compare to the Wald estimates in table3?

Write Tables to Two Files. Display a directory of the 11 saved regressions. 

In [1]:
cd "~/Eco 727 - Research Methods/Analysis"

C:\Users\Rafael\Eco 727 - Research Methods\Analysis


## A. Install the estout Package

In [2]:
//ssc install estout, replace

## B. Use the ``Birth Quarter.dta'' data

In [1]:
use "..\Data\Birth Quarter" if inlist(cohort, 4) & inlist(census, 80)

In [2]:
describe
summarize



Contains data from ..\Data\Birth Quarter.dta
  obs:       486,926                          
 vars:            12                          18 Mar 2019 13:13
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
age             byte    %8.0g                 
ageq            float   %9.0g                 
educ            byte    %8.0g                 
lwklywge        float   %9.0g                 
married         byte    %8.0g                 
census          byte    %8.0g                 
qob             byte    %8.0g                 
race            byte    %8.0g                 
smsa      

We limit the sample to the 1940s birth cohorts and confirm that the observations in the data are from
the 1980 Census. Moreover, we ensure that we have 486,926 observations, matching the sample size in Angrist and Krueger's
table 6.

In [3]:
tab educ


       educ |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        504        0.10        0.10
          1 |        176        0.04        0.14
          2 |        344        0.07        0.21
          3 |        418        0.09        0.30
          4 |        674        0.14        0.43
          5 |      1,030        0.21        0.65
          6 |      2,429        0.50        1.14
          7 |      4,691        0.96        2.11
          8 |     11,060        2.27        4.38
          9 |     13,021        2.67        7.05
         10 |     15,996        3.29       10.34
         11 |     14,995        3.08       13.42
         12 |    168,442       34.59       48.01
         13 |     36,529        7.50       55.51
         14 |     48,585        9.98       65.49
         15 |     21,869        4.49       69.98
         16 |     69,234       14.22       84.20
         17 |     23,871        4.90       89.10
         18 |     2

#### Generating the necessary variables

In [5]:
/*Generating black*/
generate black = (race == 1)
tab black




      black |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |    447,171       91.84       91.84
          1 |     39,755        8.16      100.00
------------+-----------------------------------
      Total |    486,926      100.00


In [6]:
/*Generating married */
generate marry = (married==1)
tab marry




      marry |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |     95,366       19.59       19.59
          1 |    391,560       80.41      100.00
------------+-----------------------------------
      Total |    486,926      100.00


In [7]:
/*Generating center center city */
generate center_city = (smsa ==1)
tab center_city




center_city |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |    396,605       81.45       81.45
          1 |     90,321       18.55      100.00
------------+-----------------------------------
      Total |    486,926      100.00


In [8]:
/*Generating age squared */
generate ageq_squared = ageq*ageq

## C. Estimate the log-Wage Regression by OLS and 2SLS

### Regress the weekly wage variable (in logs) on education.

In [9]:
eststo: reg lwklywge educ 


      Source |       SS           df       MS      Number of obs   =   486,926
-------------+----------------------------------   F(1, 486924)    =  34137.04
       Model |  13286.5814         1  13286.5814   Prob > F        =    0.0000
    Residual |  189517.161   486,924  .389213022   R-squared       =    0.0655
-------------+----------------------------------   Adj R-squared   =    0.0655
       Total |  202803.743   486,925  .416498932   Root MSE        =    .62387

------------------------------------------------------------------------------
    lwklywge |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        educ |   .0554276      .0003   184.76   0.000     .0548397    .0560156
       _cons |    5.04521   .0041685  1210.33   0.000      5.03704    5.053381
------------------------------------------------------------------------------
(est1 stored)


We can see that one more year of education leads to a 5.54% percent increase in wages. 

### Produce birth-quarter-1 Wald estimates

In [10]:
/*Generating first quarter */
generate first_quarter = (qob == 1)
tab first_quarter




first_quart |
         er |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |    369,206       75.82       75.82
          1 |    117,720       24.18      100.00
------------+-----------------------------------
      Total |    486,926      100.00


### Wald estimates using rise over run

In [11]:
tabstat lwklywge educ, by(first_quarter)


Summary statistics: mean
  by categories of: first_quarter 

first_quarter |  lwklywge      educ
--------------+--------------------
            0 |  5.797265   13.5917
            1 |  5.798075  13.50922
--------------+--------------------
        Total |  5.797461  13.57176
-----------------------------------


In [12]:
display "Wald Estimate (slope) = "(5.798075-5.797265) / (13.50922-13.5917)

Wald Estimate (slope) = -.00982056


### Wald estimates using 2SLS

In [13]:
eststo:ivregress 2sls lwklywge (educ = first_quarter), first


First-stage regressions
-----------------------

                                                Number of obs     =    486,926
                                                F(   1, 486924)   =      68.38
                                                Prob > F          =     0.0000
                                                R-squared         =     0.0001
                                                Adj R-squared     =     0.0001
                                                Root MSE          =     2.9800

-------------------------------------------------------------------------------
         educ |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
first_quarter |  -.0824827   .0099745    -8.27   0.000    -.1020324    -.062933
        _cons |    13.5917   .0049044  2771.33   0.000     13.58209    13.60131
---------------------------------------------------------------------------

Based on the Wald estimates, one more year of education leads to a .98% (almost a 1%) decrease in wages. This is similar to the wald estimates that we found using the rise over run method. 

### Estimate this log-wage regression by 2SLS with birth-quarter instruments; include birth-year effects (with 1949 as the base year)

In [14]:
eststo:ivregress 2sls lwklywge (educ = ib4.qob) ib49.yob, first 
// ib4 with the 4th quarter as base quarter. And ib49 with 1949 as base year.


First-stage regressions
-----------------------

                                                Number of obs     =    486,926
                                                F(  12, 486913)   =     239.55
                                                Prob > F          =     0.0000
                                                R-squared         =     0.0059
                                                Adj R-squared     =     0.0058
                                                Root MSE          =     2.9715

------------------------------------------------------------------------------
        educ |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         yob |
         40  |  -.6318383    .019617   -32.21   0.000    -.6702871   -.5933895
         41  |  -.5358309   .0195909   -27.35   0.000    -.5742284   -.4974334
         42  |  -.3647637   .0187076   -19.50   0.000      -.4014

### D. Replicate Angrist and Krueger's Table 6

First, we compute the OLS regressions in columns 1, 3, 5, 7. Notice that in each regression we include birth-year effects (with 1949 as the base year).

In [15]:
/* Column 1 */
eststo: regress lwklywge educ ib49.yob 

/* Column 3 */
eststo: regress lwklywge educ ib49.yob ageq ageq_squared

/* Column 5 */
eststo: regress lwklywge educ black center_city marry ib49.yob i.region

/* Column 7 */
eststo: regress lwklywge educ black center_city marry ib49.yob i.region ageq ageq_squared



      Source |       SS           df       MS      Number of obs   =   486,926
-------------+----------------------------------   F(10, 486915)   =   4396.64
       Model |  16795.7577        10  1679.57577   Prob > F        =    0.0000
    Residual |  186007.985   486,915  .382013257   R-squared       =    0.0828
-------------+----------------------------------   Adj R-squared   =    0.0828
       Total |  202803.743   486,925  .416498932   Root MSE        =    .61807

------------------------------------------------------------------------------
    lwklywge |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        educ |   .0573383   .0002981   192.37   0.000     .0567541    .0579224
             |
         yob |
         40  |   .2510504   .0040846    61.46   0.000     .2430446    .2590561
         41  |   .2426493    .004078    59.50   0.000     .2346565    .2506421
         42  |   .2

West-North~l  |   .0226072   .0047785     4.73   0.000     .0132415    .0319729
South Atla..  |  -.0123754   .0042062    -2.94   0.003    -.0206195   -.0041314
East-South~l  |  -.0276449   .0049809    -5.55   0.000    -.0374072   -.0178825
West-South~l  |   .0345582   .0045556     7.59   0.000     .0256293     .043487
    Mountain  |   .0250948   .0052438     4.79   0.000      .014817    .0353726
     Pacific  |   .0926402   .0043443    21.32   0.000     .0841256    .1011548
              |
         ageq |   .1517973   .0379337     4.00   0.000     .0774484    .2261462
 ageq_squared |  -.0019453   .0005448    -3.57   0.000    -.0030131   -.0008774
        _cons |   1.937132   .6537097     2.96   0.003     .6558818    3.218383
-------------------------------------------------------------------------------
(est7 stored)


With a few exceptions, the estimates match those in table 6. AK report the wrong signs for their SMSA(1 = center city) effects.

We then compute 2sls regressions in columns 2, 4, 6, 8. We do so in order to have an easier time inspecting the instruments. Notice that in each regression we include birth-quarter/birth-year interactions. We set the 4th quarter as base quarter and 1949 as base year. 

In [16]:
/* Column 2 */
eststo: ivregress 2sls lwklywge (educ =ib4.qob#ib49.yob) ib49.yob, first 
estat firststage

/* Column 4 */
eststo: ivregress 2sls lwklywge (educ =ib4.qob#ib49.yob) ib49.yob ageq ageq_squared, first 
estat firststage

/* Column 6 */
eststo: ivregress 2sls lwklywge (educ =ib4.qob#ib49.yob) black center_city marry i.region ib49.yob, first 
estat firststage

/* Column 8 */
eststo: ivregress 2sls lwklywge (educ =ib4.qob#ib49.yob) black center_city marry i.region ageq ageq_squared ib49.yob, first 
estat firststage




First-stage regressions
-----------------------

                                                Number of obs     =    486,926
                                                F(  39, 486886)   =      77.36
                                                Prob > F          =     0.0000
                                                R-squared         =     0.0062
                                                Adj R-squared     =     0.0061
                                                Root MSE          =     2.9712

------------------------------------------------------------------------------
        educ |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         yob |
         40  |  -.4790786   .0394497   -12.14   0.000    -.5563987   -.4017585
         41  |  -.4573497   .0390959   -11.70   0.000    -.5339764    -.380723
         42  |  -.1900694   .0363524    -5.23   0.000     -.2613

        educ |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         yob |
         40  |   2.813559   6.853902     0.41   0.681    -10.61988    16.24699
         41  |   2.079454    5.32198     0.39   0.696    -8.351462    12.51037
         42  |   1.688396   3.984986     0.42   0.672    -6.122053    9.498844
         43  |   1.055058   2.843132     0.37   0.711    -4.517391    6.627508
         44  |   .6574859   1.896678     0.35   0.729    -3.059945    4.374916
         45  |   .3511506   1.146299     0.31   0.759    -1.895559     2.59786
         46  |   .3349667   .5934248     0.56   0.572    -.8281274    1.498061
         47  |   .0775279    .243023     0.32   0.750    -.3987896    .5538455
         48  |   .0042465   .1003984     0.04   0.966    -.1925313    .2010243
             |
        ageq |   3.022153   6.011628     0.50   0.615     -8.76045    14.80476
ageq_squared |  -.0487

East-North~l  |  -.4725709   .0198763   -23.78   0.000    -.5115279   -.4336139
West-North~l  |  -.0981649   .0230482    -4.26   0.000    -.1433387   -.0529912
South Atla..  |  -.5077498   .0202754   -25.04   0.000    -.5474889   -.4680107
East-South~l  |  -.9141151   .0239891   -38.11   0.000     -.961133   -.8670973
West-South~l  |  -.3206177   .0219703   -14.59   0.000    -.3636788   -.2775566
    Mountain  |   .2955154   .0252892    11.69   0.000     .2459493    .3450815
     Pacific  |   .3265227   .0209485    15.59   0.000     .2854644    .3675811
              |
          yob |
          40  |  -.4760834   .0385758   -12.34   0.000    -.5516908   -.4004761
          41  |  -.4605596   .0382303   -12.05   0.000    -.5354898   -.3856294
          42  |  -.2229276   .0355494    -6.27   0.000    -.2926033   -.1532519
          43  |  -.2749167   .0365811    -7.52   0.000    -.3466145    -.203219
          44  |  -.2144935   .0365075    -5.88   0.000    -.2860471   -.1429398
        

  2SLS Size of nominal 5% Wald test  |  86.17   44.78   30.72   23.65
  LIML Size of nominal 5% Wald test  |   3.88    2.18    1.89    1.75
  ---------------------------------------------------------------------


note: 3.qob#48.yob dropped due to collinearity
note: 3.qob#49b.yob dropped due to collinearity

First-stage regressions
-----------------------

                                                Number of obs     =    486,926
                                                F(  50, 486875)   =     517.34
                                                Prob > F          =     0.0000
                                                R-squared         =     0.0504
                                                Adj R-squared     =     0.0504
                                                Root MSE          =     2.9042

-------------------------------------------------------------------------------
         educ |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
------

               41.yob 42.yob 43.yob 44.yob 45.yob 46.yob 47.yob 48.yob
               1.qob#40.yob 1.qob#41.yob 1.qob#42.yob 1.qob#43.yob
               1.qob#44.yob 1.qob#45.yob 1.qob#46.yob 1.qob#47.yob
               1.qob#48.yob 1.qob#49b.yob 2.qob#40.yob 2.qob#41.yob
               2.qob#42.yob 2.qob#43.yob 2.qob#44.yob 2.qob#45.yob
               2.qob#46.yob 2.qob#47.yob 2.qob#48.yob 2.qob#49b.yob
               3.qob#40.yob 3.qob#41.yob 3.qob#42.yob 3.qob#43.yob
               3.qob#44.yob 3.qob#45.yob 3.qob#46.yob 3.qob#47.yob
(est11 stored)


  First-stage regression summary statistics
  --------------------------------------------------------------------------
               |            Adjusted      Partial
      Variable |   R-sq.       R-sq.        R-sq.   F(28,486875)  Prob > F
  -------------+------------------------------------------------------------
          educ |  0.0504      0.0504       0.0002        2.7352    0.0000
  ------------------------------------------

The F statistic or eigenvalue are all low  indicating a weak instruments problem in each specification.

### E. Compare the Estimates

### How does the Wald estimate in Part C compare to the Wald estimates in table3?

In [17]:
 ivregress 2sls lwklywge (educ = first_quarter)


Instrumental variables (2SLS) regression          Number of obs   =    486,926
                                                  Wald chi2(1)    =       0.14
                                                  Prob > chi2     =     0.7111
                                                  R-squared       =          .
                                                  Root MSE        =     .65347

------------------------------------------------------------------------------
    lwklywge |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        educ |  -.0098222   .0265177    -0.37   0.711    -.0617959    .0421516
       _cons |   5.930765   .3598933    16.48   0.000     5.225387    6.636143
------------------------------------------------------------------------------
Instrumented:  educ
Instruments:   first_quarter


The Wald estimate in part c, represents a negative .98%(almost 1%) return on schooling, while the Wald estimates in table 3 represents a 10.2% return on schooling.

### How does the 2SLS estimate with three birth-quarter instruments compare to the 2SLS estimates with birth-quarter × birth-year instruments?

In [19]:
quietly ivregress 2sls lwklywge (educ = ib4.qob) ib49.yob
display _b[educ]

quietly  ivregress 2sls lwklywge ib49.yob (educ =ib4.qob#ib49.yob)
display _b[educ]



-.06119036


.05529783


Based on the return to schooling, the second model is more favorable, as it offers a higher return to schooling. However, one must inspect the birth-quarter x birth year instruments and see for weaknesses.

### Are the over-identifying restrictions rejected in the models with (a) three birth-quarter instruments or (b) 29 birth-quarter/birth-year instruments?

In [19]:
quietly ivregress 2sls lwklywge (educ = ib4.qob) ib49.yob, first
estat overid 
estat firststage




  Tests of overidentifying restrictions:

  Sargan (score) chi2(2) =   10.889  (p = 0.0043)
  Basmann chi2(2)        =   10.889  (p = 0.0043)


  First-stage regression summary statistics
  --------------------------------------------------------------------------
               |            Adjusted      Partial
      Variable |   R-sq.       R-sq.        R-sq.   F(3,486913)   Prob > F
  -------------+------------------------------------------------------------
          educ |  0.0059      0.0058       0.0002       28.9475    0.0000
  --------------------------------------------------------------------------


  Minimum eigenvalue statistic = 28.9475     

  Critical Values                      # of endogenous regressors:    1
  Ho: Instruments are weak             # of excluded instruments:     3
  ---------------------------------------------------------------------
                                     |    5%     10%     20%     30%
  2SLS relative bias                 |  13.91

Here we test the null hypothesis that all three estimates are equal. We do reject the null hypothesis. Furthermore, we find the eigenvalue stat to be high, showing that there is not a weak instruments problem for model (a). 

In [20]:
quietly ivregress 2sls lwklywge ib49.yob (educ =ib4.qob#ib49.yob), first 
estat overid 
estat firststage




  Tests of overidentifying restrictions:

  Sargan (score) chi2(29)=  101.621  (p = 0.0000)
  Basmann chi2(29)       =  101.634  (p = 0.0000)


  First-stage regression summary statistics
  --------------------------------------------------------------------------
               |            Adjusted      Partial
      Variable |   R-sq.       R-sq.        R-sq.   F(30,486886)  Prob > F
  -------------+------------------------------------------------------------
          educ |  0.0062      0.0061       0.0005       7.62315    0.0000
  --------------------------------------------------------------------------


  Minimum eigenvalue statistic = 7.62315     

  Critical Values                      # of endogenous regressors:    1
  Ho: Instruments are weak             # of excluded instruments:    30
  ---------------------------------------------------------------------
                                     |    5%     10%     20%     30%
  2SLS relative bias                 |  21.42

Again we reject the null hypothesis, findind that not all estimates are equal. However this time, we find the eigenvalue stat to be low, showing that there may is a weak instruments problem for model (b).

## F. Write Tables to Two Files

### Display a directory of the 11 saved regressions

In [21]:
eststo dir


--------------------------------------------------------------------------------
        name | command      depvar       npar  title 
-------------+------------------------------------------------------------------
        est1 | regress      lwklywge        2  Linear regression
        est2 | ivregress    lwklywge        2  Instrumental variables (2SLS)
             |                                 regression
        est3 | ivregress    lwklywge       12  Instrumental variables (2SLS)
             |                                 regression
        est4 | regress      lwklywge       12  Linear regression
        est5 | regress      lwklywge       14  Linear regression
        est6 | regress      lwklywge       24  Linear regression
        est7 | regress      lwklywge       26  Linear regression
        est8 | ivregress    lwklywge       12  Instrumental variables (2SLS)
             |                                 regression
        est9 | ivregress    lwklywge       14  Instru

### Have esttab write the estimatesfrom the three regressionsin Part C

In [5]:
esttab est1 est2 est3 using Results/Exercise4_table_a.tex, replace nodepvars nostar noconstant booktabs /// 
label title("WALD ESTIMATES OF THE RETURN TO EDUCATION FOR MEN BORN 1940-1949: 1980 CENSUS") ///
mtitles("OLS" "TSLS" "TSLS") ///
b(4) se(4) stats(N r2, fmt(%3.0f %4.3f) labels("N" "R-sq")) ///
coeflabels(  educ "Years of Education") ///
indicate("9 Year-of-birth-dummies = *.yob") alignment(r) ///
addnotes("The dependent variables is the log of weekly earnings." ///
"Sample consist of males born in the United States." /// 
"Estimates in (2) are instrumented with samples born in 1st quarter of year only." ///
"Estimates in (3) are instrumented with a full set of birth quarters." ///
"The sample is drawn from the 5% samples of the 1980 Census.")


estimation result est1 not found


r(111);


#### In case my table won't show, here is a  representation.

In [42]:
esttab est1 est2 est3, replace nodepvars nostar noconstant /// 
label title("WALD ESTIMATES OF THE RETURN TO EDUCATION FOR MEN BORN 1940-1949: 1980 CENSUS") ///
mtitles("OLS" "TSLS" "TSLS") ///
b(4) se(4) stats(N r2, fmt(%3.0f %4.3f) labels("N" "R-sq")) ///
coeflabels(  educ "Years of Education") ///
indicate("9 Year-of-birth-dummies = *.yob") alignment(r) ///
addnotes("The dependent variables is the log of weekly earnings." ///
"Sample consist of males born in the United States." /// 
"Estimates in (2) are instrumented with samples born in 1st quarter of year only." ///
"Estimates in (3) are instrumented with a full set of birth quarters." ///
"The sample is drawn from the 5% samples of the 1980 Census.")



WALD ESTIMATES OF THE RETURN TO EDUCATION FOR MEN BORN 1940-1949: 1980 CENSUS
-----------------------------------------------------------
                              (1)          (2)          (3)
                              OLS         TSLS         TSLS
-----------------------------------------------------------
Years of Education         0.0554      -0.0098      -0.0612
                         (0.0003)     (0.0265)     (0.0257)

9 Year-of-birth-du~s           No           No          Yes
-----------------------------------------------------------
N                          486926       486926       486926
R-sq                        0.066            .            .
-----------------------------------------------------------
Standard errors in parentheses
The dependent variables is the log of weekly earnings.
Sample consist of males born in the United States.
Estimates in (2) are instrumented with samples born in 1st quarter of year only.
Estimates in (3) are instrumented with a f

## Have esttab write the estimates from regressions 5-8 (from table 6) in Part D

Since ols and 2sls regressions were done seperately by blocks, the estab's won't be in order but rather alternating.  

In [44]:
esttab est6 est10 est7 est11 using Results/Exercise4_table_b.tex, replace nodepvars nostar noconstant booktabs ///
label title("OLS & TSLS ESTIMATES OF THE RETURN TO EDUCATION FOR MEN BORN 1940-1949") ///
mtitles("OLS" "TSLS" "OLS" "TSLS") ///
b(4) se(4) stats(N r2, fmt(%3.0f %4.3f) labels("N" "R-sq")) ///
coeflabels(  educ "Years of Education" black "Race (1= black)" center_city "SMSA(1= center city)" marry "Married (1= married)" ///
ageq "Age" ageq_squared "Age-Squared") ///
indicate("9 Year-of-birth-dummies = *.yob" "8 Region-of-residence-dummies = *.region") alignment(r) ///
addnotes("The dependent variables is the log of weekly earnings." ///
"Sample consist of males born in the United States." /// 
"The sample is drawn from the 5% samples of the 1980 Census." /// 
"Instrument are a full set of quarter-of-birth times year-of-birth interactions.")

(output written to Results/Exercise4_table_b.tex)


#### Again, in case my table doesn't show, here is a representation.

In [46]:
esttab est6 est10 est7 est11, replace nodepvars nostar noconstant ///
label title("OLS & TSLS ESTIMATES OF THE RETURN TO EDUCATION FOR MEN BORN 1940-1949") ///
mtitles("OLS" "TSLS" "OLS" "TSLS") ///
b(4) se(4) stats(N r2, fmt(%3.0f %4.3f) labels("N" "R-sq")) ///
coeflabels(  educ "Years of Education" black "Race (1= black)" center_city "SMSA(1= center city)" marry "Married (1= married)" ///
ageq "Age" ageq_squared "Age-Squared") ///
indicate("9 Year-of-birth-dummies = *.yob" "8 Region-of-residence-dummies = *.region") alignment(r) ///
addnotes("The dependent variables is the log of weekly earnings." ///
"Sample consist of males born in the United States." /// 
"The sample is drawn from the 5% samples of the 1980 Census." /// 
"Instrument are a full set of quarter-of-birth times year-of-birth interactions.")


OLS & TSLS ESTIMATES OF THE RETURN TO EDUCATION FOR MEN BORN 1940-1949
------------------------------------------------------------------------
                              (1)          (2)          (3)          (4)
                              OLS         TSLS          OLS         TSLS
------------------------------------------------------------------------
Years of Education         0.0520       0.0393       0.0521       0.0779
                         (0.0003)     (0.0145)     (0.0003)     (0.0239)

Race (1= black)           -0.2107      -0.2266      -0.2108      -0.1786
                         (0.0032)     (0.0183)     (0.0032)     (0.0299)

SMSA(1= center city)      -0.1418      -0.1535      -0.1419      -0.1182
                         (0.0023)     (0.0135)     (0.0023)     (0.0220)

Married (1= married)       0.2445       0.2442       0.2444       0.2450
                         (0.0022)     (0.0022)     (0.0022)     (0.0023)

Age                                             